In [17]:
"""
CELL 1: SETUP
Description: Import required libraries, load configuration, and load prompts
"""

# Load configuration
%run vyapar_config.ipynb

# Load prompts library
%run vaani_prompts.ipynb

# Additional imports for AI functions
import openai
from anthropic import Anthropic
import time
from typing import List, Dict, Tuple, Optional

print("✅ vaani_functions.ipynb loaded")
print("✅ Config available via get_item(), get_api_key(), etc.")
print("✅ Prompts library loaded")

✅ Libraries imported successfully
✅ Master Registry Link configured
📍 Link: https://docs.google.com/spreadsheets/d/e/2PACX-1vQdOVYDNLuMG...

🔄 Testing registry load...
✅ Registry loaded successfully: 8 items found

📋 Available items in registry:
  1. default_model
  2. openai_api_key
  3. anthropic_api_key
  4. master_registry_link
  5. usage_data
  6. dropoff_analysis
  7. user_research
  8. excel_db_1000_items

🧪 Testing get_item() function:

1. Testing with 'default_model':
✅ Registry loaded successfully: 8 items found
✅ Retrieved 'default_model'
   Result: {'model': 'claude-sonnet-4-20250514', 'temperature': 0.3, 'max_tokens': 1000, 'provider': 'anthropic'}

2. Testing with non-existent item:
✅ Registry loaded successfully: 8 items found
❌ Item 'this_does_not_exist' not found in registry
💡 Available items: default_model, openai_api_key, anthropic_api_key, master_registry_link, usage_data, dropoff_analysis, user_research, excel_db_1000_items

3. Testing with Sarvam API (might not ha

In [36]:
"""
CELL 2: OPENAI CLIENT SETUP
Description: Create OpenAI client using API key from registry
"""

def get_openai_client():
    """
    Initialize and return OpenAI client.
    Uses API key from registry.
    """
    api_key = get_api_key('openai_api_key')
    
    if not api_key or 'YOUR' in api_key:
        print("⚠️ OpenAI API key not configured properly")
        return None
    
    client = openai.OpenAI(api_key=api_key)
    print("✅ OpenAI client initialized")
    return client

# Test it
# openai_client = get_openai_client()

In [37]:
"""
CELL 3: ANTHROPIC CLIENT SETUP
Description: Create Anthropic client using API key from registry
"""

def get_anthropic_client():
    """
    Initialize and return Anthropic client.
    Uses API key from registry.
    """
    api_key = get_api_key('anthropic_api_key')
    
    if not api_key or 'YOUR' in api_key:
        print("⚠️ Anthropic API key not configured properly")
        return None
    
    client = Anthropic(api_key=api_key)
    print("✅ Anthropic client initialized")
    return client

# Test it
# anthropic_client = get_anthropic_client()

In [11]:
"""
CELL 4: AI AGENT 1 - BASIC EXTRACTOR
Description: Extracts amount, item name, and category from voice input
Use case: Quick, simple extractions
"""

def agent_basic_extractor(voice_input: str, model: str = None) -> Dict:
    """
    Basic extraction agent: Extracts amount, item, category from voice input.
    
    Args:
        voice_input: The voice transcription (e.g., "chai samosa 140 rupees")
        model: Model to use (defaults to registry default_model)
    
    Returns:
        {
            'amount': 140,
            'item': 'Chai Samosa',
            'category': 'Food',
            'raw_response': '...',
            'model_used': 'gpt-4o-mini',
            'time_taken': 1.2
        }
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Create prompt
    prompt = f"""You are an expense tracking assistant. Extract the following from the voice input:

Voice Input: "{voice_input}"

Extract and return ONLY a JSON object with these fields:
- amount: The amount spent (number only, no currency)
- item: The item name
- category: Best category for this expense (Food, Transport, Utilities, etc.)

Return ONLY valid JSON, nothing else."""

    # Call LLM
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            # OpenAI call
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            # Anthropic call
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Parse JSON response
        import json
        extracted = json.loads(raw_response)
        
        # Add metadata
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        
        return extracted
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Basic Extractor Agent:")
test_result = agent_basic_extractor("chai samosa 140 rupees")
print(test_result)

🧪 Testing Basic Extractor Agent:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
{'error': 'OpenAI client not available'}


In [12]:
"""
CELL 5: AI AGENT 2 - SMART CATEGORIZER
Description: Uses context from Excel DB for more accurate categorization
Use case: When you want high accuracy categorization
"""

def agent_smart_categorizer(voice_input: str, excel_db_context: str = None, model: str = None) -> Dict:
    """
    Smart categorization agent: Better category matching using Excel DB context.
    
    Args:
        voice_input: The voice transcription
        excel_db_context: Sample categories from Excel DB (optional)
        model: Model to use (defaults to registry default_model)
    
    Returns:
        Same as agent_basic_extractor but with improved categorization
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Add context to prompt if provided
    context_section = ""
    if excel_db_context:
        context_section = f"\n\nCommon categories used by similar businesses:\n{excel_db_context}\n"
    
    # Create enhanced prompt
    prompt = f"""You are an expense tracking assistant for Indian MSMEs. Extract information from this voice input.

Voice Input: "{voice_input}"
{context_section}
Extract and return ONLY a JSON object:
- amount: Amount spent (number only)
- item: Item name
- category: Most appropriate category (consider common Indian business expenses)

Return ONLY valid JSON."""

    # Call LLM (same logic as basic extractor)
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Parse JSON
        import json
        extracted = json.loads(raw_response)
        
        # Add metadata
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        extracted['used_context'] = excel_db_context is not None
        
        return extracted
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Smart Categorizer Agent:")
test_result = agent_smart_categorizer("petrol 500 rupees")
print(test_result)

🧪 Testing Smart Categorizer Agent:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
{'error': 'OpenAI client not available'}


In [13]:
"""
CELL 6: AI AGENT 3 - MULTI-ITEM HANDLER
Description: Handles voice inputs with multiple items
Use case: "chai 60 rupees, samosa 80 rupees"
"""

def agent_multi_item_handler(voice_input: str, model: str = None) -> Dict:
    """
    Multi-item handler: Extracts multiple items from single voice input.
    
    Args:
        voice_input: Voice input with multiple items
        model: Model to use
    
    Returns:
        {
            'items': [
                {'amount': 60, 'item': 'Chai', 'category': 'Food'},
                {'amount': 80, 'item': 'Samosa', 'category': 'Food'}
            ],
            'total_amount': 140,
            'item_count': 2,
            'model_used': 'gpt-4o-mini',
            'time_taken': 1.5
        }
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Create prompt
    prompt = f"""You are an expense tracking assistant. The voice input may contain multiple items.

Voice Input: "{voice_input}"

Extract ALL items mentioned. Return ONLY a JSON object:
{{
    "items": [
        {{"amount": number, "item": "name", "category": "category"}},
        ...
    ],
    "total_amount": sum of all amounts
}}

Return ONLY valid JSON."""

    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 800)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 800)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Parse JSON
        import json
        extracted = json.loads(raw_response)
        
        # Add metadata
        extracted['item_count'] = len(extracted.get('items', []))
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        
        return extracted
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Multi-Item Handler Agent:")
test_result = agent_multi_item_handler("chai 60 rupees, samosa 80 rupees")
print(test_result)

🧪 Testing Multi-Item Handler Agent:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
{'error': 'OpenAI client not available'}


In [14]:
"""
CELL 7: AGENT ROUTER
Description: Intelligently routes to the right agent based on input complexity
"""

def route_to_agent(voice_input: str, model: str = None) -> Dict:
    """
    Smart router: Analyzes input and calls the appropriate agent.
    
    Logic:
    - If multiple items detected → Multi-Item Handler
    - If simple input → Basic Extractor
    - If needs context → Smart Categorizer
    
    Args:
        voice_input: The voice input
        model: Model to use (optional)
    
    Returns:
        Result from the chosen agent + routing metadata
    """
    # Simple heuristics for routing
    voice_lower = voice_input.lower()
    
    # Check for multiple items (commas, "and", multiple numbers)
    has_comma = ',' in voice_input
    has_and = ' and ' in voice_lower or ' aur ' in voice_lower
    number_count = sum(c.isdigit() for c in voice_input)
    
    if has_comma or has_and or number_count > 6:
        # Route to multi-item handler
        print("🎯 Routing to: Multi-Item Handler")
        result = agent_multi_item_handler(voice_input, model)
        result['agent_used'] = 'multi_item_handler'
    
    else:
        # Route to basic extractor
        print("🎯 Routing to: Basic Extractor")
        result = agent_basic_extractor(voice_input, model)
        result['agent_used'] = 'basic_extractor'
    
    return result

# Test it
print("\n🧪 Testing Agent Router:")
print("\n1. Single item:")
result1 = route_to_agent("chai 60 rupees")
print(f"Result: {result1}\n")

print("2. Multiple items:")
result2 = route_to_agent("chai 60 rupees, samosa 80 rupees")
print(f"Result: {result2}")


🧪 Testing Agent Router:

1. Single item:
🎯 Routing to: Basic Extractor
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
Result: {'error': 'OpenAI client not available', 'agent_used': 'basic_extractor'}

2. Multiple items:
🎯 Routing to: Multi-Item Handler
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
Result: {'error': 'OpenAI client not available', 'agent_used': 'multi_item_handler'}


In [15]:
"""
CELL 8: UTILITY FUNCTIONS
Description: Helper functions for formatting, validation, etc.
"""

def format_amount(amount: any) -> float:
    """Convert amount to float, handle different formats"""
    try:
        if isinstance(amount, str):
            # Remove currency symbols, commas
            amount = amount.replace('₹', '').replace(',', '').strip()
        return float(amount)
    except:
        return None

def validate_extraction(result: Dict) -> Tuple[bool, List[str]]:
    """
    Validate extraction result.
    Returns: (is_valid, list_of_errors)
    """
    errors = []
    
    if 'error' in result:
        errors.append(f"API Error: {result['error']}")
        return False, errors
    
    # Check required fields
    if 'amount' not in result:
        errors.append("Missing amount")
    elif not isinstance(result['amount'], (int, float)):
        errors.append("Amount is not a number")
    
    if 'item' not in result:
        errors.append("Missing item name")
    elif not result['item']:
        errors.append("Item name is empty")
    
    return len(errors) == 0, errors

# Test utilities
print("🧪 Testing Utility Functions:")
print(f"Format '₹1,500': {format_amount('₹1,500')}")
print(f"Validate good result: {validate_extraction({'amount': 100, 'item': 'Chai'})}")
print(f"Validate bad result: {validate_extraction({'error': 'Failed'})}")

🧪 Testing Utility Functions:
Format '₹1,500': 1500.0
Validate good result: (True, [])
Validate bad result: (False, ['API Error: Failed'])


In [38]:
"""
CELL 9: SUMMARY
Description: Shows all available functions in this notebook
"""

print("""
📚 VAANI FUNCTIONS AVAILABLE
=====================================================

🤖 AI AGENTS:
1. agent_basic_extractor(voice_input, model=None)
   - Simple, fast extraction
   
2. agent_smart_categorizer(voice_input, excel_db_context=None, model=None)
   - Better categorization with context
   
3. agent_multi_item_handler(voice_input, model=None)
   - Handles multiple items in one input
   
4. route_to_agent(voice_input, model=None)
   - Smart router (RECOMMENDED)

🛠️ UTILITIES:
- format_amount(amount) - Clean amount formatting
- validate_extraction(result) - Validate extraction results

🔌 CLIENTS:
- get_openai_client() - Get OpenAI client
- get_anthropic_client() - Get Anthropic client

=====================================================
✅ All functions loaded and ready!
""")


📚 VAANI FUNCTIONS AVAILABLE

🤖 AI AGENTS:
1. agent_basic_extractor(voice_input, model=None)
   - Simple, fast extraction
   
2. agent_smart_categorizer(voice_input, excel_db_context=None, model=None)
   - Better categorization with context
   
3. agent_multi_item_handler(voice_input, model=None)
   - Handles multiple items in one input
   
4. route_to_agent(voice_input, model=None)
   - Smart router (RECOMMENDED)

🛠️ UTILITIES:
- format_amount(amount) - Clean amount formatting
- validate_extraction(result) - Validate extraction results

🔌 CLIENTS:
- get_openai_client() - Get OpenAI client
- get_anthropic_client() - Get Anthropic client

✅ All functions loaded and ready!



In [40]:
"""
CELL 9A: CATEGORY HELPER FUNCTIONS
Description: Load and format consolidated categories
"""

def load_categories_data() -> pd.DataFrame:
    """
    Load consolidated categories from registry.
    
    Returns:
        DataFrame with category data
    """
    try:
        link = get_data_file_link('consolidated_categories')
        if not link:
            print("⚠️ Categories not found in registry")
            return None
        
        df = pd.read_excel(link)
        print(f"✅ Loaded {len(df)} categories")
        return df
    except Exception as e:
        print(f"❌ Error loading categories: {e}")
        return None


def format_categories_for_prompt(df: pd.DataFrame = None, top_n: int = 50) -> str:
    """
    Format categories as text for inclusion in prompts.
    
    Args:
        df: DataFrame with categories (loads if None)
        top_n: Number of top categories to include
    
    Returns:
        Formatted string for prompt
    """
    if df is None:
        df = load_categories_data()
        if df is None:
            return ""
    
    categories_text = ""
    for idx, row in df.head(top_n).iterrows():
        rank = row.get('#', idx + 1)
        category = row.get('Consolidated Category', 'Unknown')
        notes = row.get('Notes', '')
        
        categories_text += f"{rank}. {category}"
        if notes:
            categories_text += f" - {notes}"
        categories_text += "\n"
    
    return categories_text.strip()


def get_top_categories(df: pd.DataFrame = None, n: int = 10) -> list:
    """
    Get list of top N category names.
    
    Args:
        df: DataFrame with categories
        n: Number to return
    
    Returns:
        List of category names
    """
    if df is None:
        df = load_categories_data()
        if df is None:
            return []
    
    return df.head(n)['Consolidated Category'].tolist()


print("✅ Category helper functions loaded")

✅ Category helper functions loaded


In [30]:
"""
CELL 9B: JSON EXTRACTION HELPER
Description: Safely extract JSON from LLM responses
"""

import re

def extract_json_from_response(response_text: str) -> str:
    """
    Extract JSON from LLM response, handling markdown code blocks and extra text.
    
    Args:
        response_text: Raw response from LLM
    
    Returns:
        Clean JSON string
    """
    # Remove markdown code blocks
    response_text = re.sub(r'```json\s*', '', response_text)
    response_text = re.sub(r'```\s*', '', response_text)
    
    # Try to find JSON object in the text
    json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
    
    if json_match:
        return json_match.group(0)
    
    return response_text.strip()

print("✅ JSON extraction helper loaded")

✅ JSON extraction helper loaded


In [31]:
"""
CELL 9C: TIMER UTILITY
Description: Clean timer wrapper for tracking cell execution time
"""

import time

class CellTimer:
    """Simple timer for tracking cell execution."""
    
    def __init__(self):
        self.start_time = time.time()
    
    def elapsed(self) -> float:
        """Get elapsed time in seconds."""
        return time.time() - self.start_time
    
    def elapsed_ms(self) -> int:
        """Get elapsed time in milliseconds."""
        return int(self.elapsed() * 1000)
    
    def print_summary(self, label: str = "CELL EXECUTION"):
        """Print formatted timer summary."""
        elapsed = self.elapsed()
        print(f"\n{'='*60}")
        print(f"⏱️ {label} TIME: {elapsed*1000:.0f} ms ({elapsed:.2f}s)")
        print(f"{'='*60}")

print("✅ Timer utility loaded")

✅ Timer utility loaded


In [32]:
"""
CELL 9D: PLAYGROUND HELPER FUNCTIONS
Description: Utilities for viewing prompts and field requirements
"""

def view_prompt(prompt_type: str):
    """
    View a system prompt.
    
    Args:
        prompt_type: 'intent', 'expense', 'sale', 'purchase', 'payment_in', 'payment_out', 'multi_item'
    """
    print("="*60)
    print(f"📋 PROMPT: {prompt_type.upper()}")
    print("="*60)
    
    prompt = get_prompt(prompt_type, text_input="[USER_INPUT_HERE]")
    print(prompt)
    print("\n" + "="*60)


def view_all_prompts():
    """View all available prompts."""
    print("="*60)
    print("📚 ALL AVAILABLE PROMPTS")
    print("="*60)
    
    prompts = list_available_prompts()
    for idx, (key, desc) in enumerate(prompts.items(), 1):
        print(f"{idx}. {key}: {desc}")
    
    print("\n💡 Usage: view_prompt('expense')")
    print("="*60)


def view_transaction_fields():
    """
    View necessary and additional fields for each transaction type.
    Based on PRD specifications.
    """
    
    fields_schema = {
        'expense': {
            'necessary': ['amount', 'item'],
            'additional': ['category', 'date', 'payment_type', 'notes'],
            'description': 'Recording business expenses'
        },
        'sale': {
            'necessary': ['customer_name', 'amount', 'items'],
            'additional': ['payment_type', 'date', 'notes', 'invoice_number'],
            'description': 'Recording sales/invoices'
        },
        'purchase': {
            'necessary': ['supplier_name', 'amount', 'items'],
            'additional': ['payment_type', 'date', 'notes', 'inventory_update'],
            'description': 'Recording purchases from suppliers'
        },
        'payment_in': {
            'necessary': ['payer_name', 'amount'],
            'additional': ['payment_type', 'date', 'notes', 'invoice_reference'],
            'description': 'Money received from customers'
        },
        'payment_out': {
            'necessary': ['payee_name', 'amount'],
            'additional': ['payment_type', 'date', 'notes', 'invoice_reference'],
            'description': 'Money paid to vendors/suppliers'
        }
    }
    
    print("="*60)
    print("📊 TRANSACTION FIELD REQUIREMENTS")
    print("="*60)
    
    for tx_type, fields in fields_schema.items():
        print(f"\n🔹 {tx_type.upper()}")
        print(f"   Description: {fields['description']}")
        print(f"   ✅ Necessary: {', '.join(fields['necessary'])}")
        print(f"   📝 Additional: {', '.join(fields['additional'])}")
    
    print("\n" + "="*60)


print("✅ Playground helper functions loaded")
print("\n💡 Available functions:")
print("   • view_prompt('expense')")
print("   • view_all_prompts()")
print("   • view_transaction_fields()")

✅ Playground helper functions loaded

💡 Available functions:
   • view_prompt('expense')
   • view_all_prompts()
   • view_transaction_fields()


In [33]:
"""
CELL 10: INTENT DETECTION AGENT (UPDATED)
Description: Identifies if input is relevant and classifies transaction type
"""

def agent_intent_detector(text_input: str, model: str = None, debug: bool = False) -> Dict:
    """
    Detects if input is relevant to VAANI and identifies transaction type.
    """
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    prompt = get_prompt('intent', text_input=text_input)
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
                max_tokens=200
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
                max_tokens=200
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        if debug:
            print(f"\n🔍 DEBUG - Intent Raw Response:")
            print(f"{raw_response}")
        
        if not raw_response or not raw_response.strip():
            return {'error': 'Empty response from model', 'model_used': model, 'time_taken': round(time_taken, 2)}
        
        import json
        clean_json = extract_json_from_response(raw_response)
        
        if debug:
            print(f"\n🔍 DEBUG - Cleaned JSON:")
            print(f"{clean_json}")
        
        intent_result = json.loads(clean_json)
        intent_result['raw_response'] = raw_response
        intent_result['model_used'] = model
        intent_result['time_taken'] = round(time_taken, 2)
        
        return intent_result
    
    except json.JSONDecodeError as e:
        return {
            'error': f'JSON parsing failed: {str(e)}',
            'raw_response': raw_response if 'raw_response' in locals() else 'No response',
            'cleaned_json': clean_json if 'clean_json' in locals() else 'No JSON',
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

print("✅ Intent detection agent loaded")

✅ Intent detection agent loaded


In [34]:
"""
CELL 11: TRANSACTION-SPECIFIC EXTRACTION AGENT (UPDATED)
Description: Extracts data using the appropriate transaction-type prompt
"""

def agent_transaction_extractor(text_input: str, transaction_type: str, model: str = None, debug: bool = False) -> Dict:
    """
    Extracts transaction details using the appropriate prompt for the transaction type.
    """
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    prompt = get_prompt(transaction_type, text_input=text_input)
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        if debug:
            print(f"\n🔍 DEBUG - Raw Response:")
            print(f"{raw_response}")
        
        if not raw_response or not raw_response.strip():
            return {
                'error': 'Empty response from model',
                'raw_response': raw_response,
                'transaction_type': transaction_type,
                'model_used': model,
                'time_taken': round(time_taken, 2)
            }
        
        import json
        clean_json = extract_json_from_response(raw_response)
        
        if debug:
            print(f"\n🔍 DEBUG - Cleaned JSON:")
            print(f"{clean_json}")
        
        extracted = json.loads(clean_json)
        extracted['transaction_type'] = transaction_type
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        
        return extracted
    
    except json.JSONDecodeError as e:
        return {
            'error': f'JSON parsing failed: {str(e)}',
            'raw_response': raw_response if 'raw_response' in locals() else 'No response',
            'cleaned_json': clean_json if 'clean_json' in locals() else 'No JSON extracted',
            'transaction_type': transaction_type,
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }
    
    except Exception as e:
        return {
            'error': f'Unexpected error: {str(e)}',
            'raw_response': raw_response if 'raw_response' in locals() else 'No response',
            'transaction_type': transaction_type,
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

print("✅ Transaction extraction agent loaded")

✅ Transaction extraction agent loaded


In [39]:
"""
CELL 12: SMART ROUTER WITH INTENT DETECTION (UPDATED)
Description: Complete flow - Intent detection → Transaction extraction OR Greeting response
"""

def route_with_intent(text_input: str, transaction_type: str = None, model: str = None) -> Dict:
    """
    Smart router with intent detection and greeting handling.
    
    Flow:
    1. If transaction_type provided → Skip intent detection, go directly to extraction
    2. If transaction_type NOT provided → Run intent detection first
    3. If greeting detected → Return friendly response
    4. If not_relevant → Return not relevant message
    5. Otherwise → Extract transaction data
    """
    result = {
        'input': text_input,
        'intent_detection_skipped': transaction_type is not None
    }
    
    # Step 1: Intent Detection (if needed)
    if transaction_type is None:
        print("🔍 Running intent detection...")
        intent_result = agent_intent_detector(text_input, model)
        
        if 'error' in intent_result:
            return {'error': f"Intent detection failed: {intent_result['error']}"}
        
        result['intent'] = intent_result
        
        # Check if relevant
        if not intent_result.get('is_relevant'):
            result['status'] = 'not_relevant'
            result['message'] = "Input is not relevant for business transactions"
            return result
        
        # Get transaction type from intent
        transaction_type = intent_result.get('transaction_type')
        
        # Handle greetings
        if transaction_type == 'greeting':
            result['status'] = 'greeting'
            result['message'] = """Hello! I'm VAANI, your voice assistant for business management. 😊

I can help you with:
✅ Recording expenses
✅ Creating sales & invoices  
✅ Managing purchases
✅ Tracking payments (in/out)

Just tell me what you'd like to do! For example:
"Add expense chai 50 rupees"
"Sharma ji bought 5kg rice for 250"
"Received payment 2000 from Kumar"

How can I help you today?"""
            return result
        
        # Handle not relevant
        if transaction_type == 'not_relevant':
            result['status'] = 'not_relevant'
            result['message'] = intent_result.get('reason', "I can only help with business transactions right now.")
            return result
    
    else:
        print(f"⏭️  Skipping intent detection, using: {transaction_type}")
        result['intent'] = {'transaction_type': transaction_type, 'skipped': True}
    
    # Step 2: Transaction-specific extraction
    print(f"📊 Extracting {transaction_type} data...")
    extraction_result = agent_transaction_extractor(text_input, transaction_type, model)
    
    if 'error' in extraction_result:
        result['error'] = f"Extraction failed: {extraction_result['error']}"
        return result
    
    result['extraction'] = extraction_result
    result['status'] = 'success'
    result['transaction_type'] = transaction_type
    
    return result

print("✅ Smart router with greeting support loaded")

✅ Smart router with greeting support loaded


In [35]:
"""
CELL 16: SUMMARY
Description: Shows all available functions
"""

print("""
📚 VAANI FUNCTIONS AVAILABLE
=====================================================

🤖 AI AGENTS:
1. agent_intent_detector(text_input, model=None, debug=False)
   - Detects transaction type
   
2. agent_transaction_extractor(text_input, transaction_type, model=None, debug=False)
   - Extracts transaction data
   
3. route_with_intent(text_input, transaction_type=None, model=None)
   - Smart router (RECOMMENDED)

🛠️ UTILITIES:
- format_amount(amount) - Clean amount formatting
- validate_extraction(result) - Validate extraction results
- extract_json_from_response(text) - Clean JSON from LLM
- CellTimer() - Track execution time

🔌 CLIENTS:
- get_openai_client() - Get OpenAI client
- get_anthropic_client() - Get Anthropic client

📋 HELPERS:
- view_prompt(prompt_type) - View a prompt
- view_all_prompts() - List all prompts
- view_transaction_fields() - Show field requirements

=====================================================
✅ All functions loaded and ready!
""")


📚 VAANI FUNCTIONS AVAILABLE

🤖 AI AGENTS:
1. agent_intent_detector(text_input, model=None, debug=False)
   - Detects transaction type
   
2. agent_transaction_extractor(text_input, transaction_type, model=None, debug=False)
   - Extracts transaction data
   
3. route_with_intent(text_input, transaction_type=None, model=None)
   - Smart router (RECOMMENDED)

🛠️ UTILITIES:
- format_amount(amount) - Clean amount formatting
- validate_extraction(result) - Validate extraction results
- extract_json_from_response(text) - Clean JSON from LLM
- CellTimer() - Track execution time

🔌 CLIENTS:
- get_openai_client() - Get OpenAI client
- get_anthropic_client() - Get Anthropic client

📋 HELPERS:
- view_prompt(prompt_type) - View a prompt
- view_all_prompts() - List all prompts
- view_transaction_fields() - Show field requirements

✅ All functions loaded and ready!



In [41]:
"""
CELL: GEMINI MODEL FUNCTIONS
"""

def get_gemini_model(model_name='gemini_2_flash'):
    """
    Get configured Gemini model from registry
    
    Args:
        model_name: 'gemini_2_flash', 'gemini_15_flash', or 'gemini_15_pro'
    
    Returns:
        Configured GenerativeModel instance
    """
    if model_name not in GEMINI_MODELS:
        print(f"⚠️ Model {model_name} not found, using gemini_2_flash")
        model_name = 'gemini_2_flash'
    
    config = GEMINI_MODELS[model_name]
    
    model = genai.GenerativeModel(
        model_name=config['model'],
        generation_config={
            'temperature': config['temperature'],
            'max_output_tokens': config['max_tokens']
        }
    )
    
    return model


def call_gemini(prompt, model_name='gemini_2_flash', return_json=True, debug=False):
    """
    Call Gemini API with prompt
    
    Args:
        prompt: The prompt text
        model_name: Which Gemini model to use
        return_json: Try to parse response as JSON
        debug: Print debug info
    
    Returns:
        Parsed JSON or raw text response
    """
    try:
        model = get_gemini_model(model_name)
        
        if debug:
            print(f"🔍 Using: {GEMINI_MODELS[model_name]['model']}")
            print(f"📝 Prompt length: {len(prompt)} chars")
        
        response = model.generate_content(prompt)
        result = response.text
        
        if debug:
            print(f"✅ Response length: {len(result)} chars")
        
        if return_json:
            # Clean JSON from response
            if '```json' in result:
                result = result.split('```json')[1].split('```')[0].strip()
            elif '```' in result:
                result = result.split('```')[1].split('```')[0].strip()
            
            try:
                return json.loads(result)
            except:
                if debug:
                    print("⚠️ Could not parse JSON, returning raw text")
                return result
        
        return result
    
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None


def gemini_intent_detector(text_input, model_name='gemini_2_flash', debug=False):
    """
    Detect transaction intent using Gemini
    
    Args:
        text_input: Voice input text
        model_name: Which model to use
        debug: Show debug info
    
    Returns:
        {"intent": "expense|sale|payment_in|payment_out|purchase|query|other"}
    """
    prompt = f"""Analyze this voice input and detect the intent.

Voice Input: "{text_input}"

Return ONLY a JSON object:
{{
    "intent": "expense or sale or payment_in or payment_out or purchase or query or other"
}}

Rules:
- expense: adding/recording expense
- sale: creating sale/invoice
- payment_in: receiving payment
- payment_out: making payment  
- purchase: buying/purchasing
- query: asking question
- other: unclear/unrelated"""
    
    result = call_gemini(prompt, model_name, return_json=True, debug=debug)
    
    if result and 'intent' in result:
        return result
    
    return {"intent": "other"}


def gemini_expense_extractor(text_input, model_name='gemini_2_flash', debug=False):
    """
    Extract expense data using Gemini
    
    Args:
        text_input: Voice input text
        model_name: Which model to use
        debug: Show debug info
    
    Returns:
        Expense data JSON
    """
    prompt = f"""Extract expense details from this voice input.

Voice Input: "{text_input}"

Return ONLY a JSON object:
{{
    "items": [
        {{
            "item_name": "name or empty string",
            "amount": number or null
        }}
    ],
    "date": "YYYY-MM-DD or null",
    "payment_type": "Cash or Online or null",
    "category": "suggested category or null"
}}

Rules:
- Extract ALL items (max 10)
- Amount as number only
- If date not mentioned, use null
- If payment not mentioned, use null"""
    
    result = call_gemini(prompt, model_name, return_json=True, debug=debug)
    return result


def gemini_with_fallback(prompt, return_json=True):
    """
    Try all Gemini models with fallback
    Order: 2.0 Flash -> 1.5 Flash -> 1.5 Pro
    """
    models = ['gemini_2_flash', 'gemini_15_flash', 'gemini_15_pro']
    
    for model in models:
        print(f"🔄 Trying {model}...")
        result = call_gemini(prompt, model, return_json)
        
        if result:
            print(f"✅ Success with {model}")
            return result
    
    print("❌ All models failed")
    return None


print("✅ Gemini functions loaded!")
print("\nAvailable:")
print("- get_gemini_model(model_name)")
print("- call_gemini(prompt, model_name, return_json, debug)")
print("- gemini_intent_detector(text_input, model_name, debug)")
print("- gemini_expense_extractor(text_input, model_name, debug)")
print("- gemini_with_fallback(prompt, return_json)")

✅ Gemini functions loaded!

Available:
- get_gemini_model(model_name)
- call_gemini(prompt, model_name, return_json, debug)
- gemini_intent_detector(text_input, model_name, debug)
- gemini_expense_extractor(text_input, model_name, debug)
- gemini_with_fallback(prompt, return_json)
